In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [3]:
address = 'Bucharest, RO'
geolocator = Nominatim(user_agent="ro_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bucharest are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bucharest are 44.4361414, 26.1027202.


In [4]:
bucharest_map=folium.Map(location=[latitude,longitude], zoom_start=12)
bucharest_map

With the following lines of code, I am importing the excel file which includes the neighborhoods in Bucharest together with their latitude and longitude and I check the first rows from the table.

In [15]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_389f077cd07b4420971ae6edcfea2559 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='clExkN5fgfAXbyi0rzKXNgJrpBL687u6nomrXt6bnVUP',
    ibm_auth_endpoint="https://iam.eu-de.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_389f077cd07b4420971ae6edcfea2559.get_object(Bucket='capstoneproject-donotdelete-pr-onxocoh4l3cfvv',Key='Bucharest_neighborhoods.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_excel(body)
df.head()


,Neighborhood,Latitude,Longitude
0,Pantelimon,44.443762,26.152474
1,Pipera,44.504337,26.136806
2,Ferentari,44.399308,26.079419
3,Bucurestii Noi,44.490262,26.032301
4,Aviatorilor,44.459504,26.081038


Let's check how many neighborhoods we have:

In [19]:
print ('The dataframe has {} neighborhoods.'.format(df.shape[0]))

The dataframe has 43 neighborhoods.


Let's see how the neighborhoods are distributed on the map:

In [24]:
map_bucharest = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bucharest)
map_bucharest

### Using Foursquare API to explore the neighborhoods

Adding inside variables the Foursquare Credentials and Version.

In [26]:
CLIENT_ID = '53EMCJS1GOQM2WGJFDZCDOP3JZOEFV5LDOS2TM4P2UUIZBB0' # your Foursquare ID
CLIENT_SECRET = '3LZWNG1ENH0E3OHSUVAA4C4ASEQEOCPAM2SGSMTWATNA3QWX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Let's make a simulation and see the closest 30 venues within a radius of 500 meters for the 5th neighborhood returned from our dataframe.

In [29]:
df.loc[4,'Neighborhood']

'Aviatorilor'

In [33]:
aviatorilor_latitude=df.loc[4, 'Latitude']
aviatorilor_longitude=df.loc[4, 'Longitude']

In [35]:
radius=500
LIMIT=30
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    aviatorilor_latitude,
    aviatorilor_longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=53EMCJS1GOQM2WGJFDZCDOP3JZOEFV5LDOS2TM4P2UUIZBB0&client_secret=3LZWNG1ENH0E3OHSUVAA4C4ASEQEOCPAM2SGSMTWATNA3QWX&v=20180605&ll=44.45950405,26.0810384064126&radius=500&limit=30'

In [36]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec11254ed78b8001b9ea2ab'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Aviatorilor',
  'headerFullLocation': 'Aviatorilor, Bucharest',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 44.4640040545, 'lng': 26.08733140782341},
   'sw': {'lat': 44.4550040455, 'lng': 26.07474540500179}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5798aa08498ea7062e7fdca8',
       'name': 'Garage Cafe',
       'location': {'address': 'Str. Barbu Ștefănescu Delavrancea nr. 7',
        'lat': 44.45775836719313,
        'lng': 26.080993954832753,
        'labeledLatLngs': [{'label': 'display',
          

Let's use this function to extract the categories of the venues:

In [37]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [38]:
venues = results['response']['groups'][0]['items']
nearby_venues1 = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns1]
# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)
# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]
nearby_venues1.head(10)

,name,categories,lat,lng
0,Garage Cafe,Coffee Shop,44.457758,26.080994
1,Pain Plaisir,Bakery,44.457005,26.080393
2,Downtown Fitness,Gym / Fitness Center,44.458576,26.077485
3,Casa Doina,Romanian Restaurant,44.458885,26.082823
4,Shaorma Orient,Middle Eastern Restaurant,44.458319,26.077610
5,Parcul Kiseleff,Park,44.457249,26.083603
6,Ristorante Il Calcio,Italian Restaurant,44.459858,26.077763
7,Piccolo Mondo,Middle Eastern Restaurant,44.459956,26.077662
8,Downtown Fitness,Gym,44.458596,26.077532
9,Supapa,Restaurant,44.455337,26.080657


How many venues were returned by Foursquare?

In [39]:
print('{} total venues were returned by Foursquare.'.format(nearby_venues1.shape[0]))

29 total venues were returned by Foursquare.


how many unique categories can be curated from all the returned venues

In [41]:
print('There are {} uniques categories.'.format(len(nearby_venues1['categories'].unique())))

There are 19 uniques categories.


##### Conclusion: There are 29 total venues in the proximity of Aviatorilor neighborhood with 19 different categories.

### Explore neighborhoods in Bucharest

A function to repeat the same process to all the neighborhoods in Bucharest:

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
bucharest_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Pantelimon
Pipera
Ferentari
Bucurestii Noi
Aviatorilor
Balta Alba
Centrul Vechi
Dristor
Colentina
Cotroceni
Crangasi
Dorobanti
Drumul Taberei
Floreasca
Gara de Nord
Ghencea
Giulesti
Grozavesti
Grivita
Ion Creanga
Militari
Obor
Primaverii
Rahova
Straulesti
Timpuri Noi
Tineretului
Unirii
Vacaresti
Victoriei
Vitan
Titan
13 septembrie
Berceni
Chitila
Iancului
Muncii
Piata Sudului
Oltenitei
Piata Romana
Piata Universitatii
Piata Resita
Stefan cel mare


The size of the resulting dataframe:

In [44]:
print(bucharest_venues.shape)
bucharest_venues.head()

(723, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Pantelimon,44.443762,26.152474,Manufaktura by Doncafé,44.442574,26.152074,Café
1,Pantelimon,44.443762,26.152474,World Class Romania,44.442656,26.152541,Gym / Fitness Center
2,Pantelimon,44.443762,26.152474,Lolly PoP Lounge & Cafe,44.442625,26.152119,Lounge
3,Pantelimon,44.443762,26.152474,Paul,44.442685,26.152064,Bakery
4,Pantelimon,44.443762,26.152474,Cinema City,44.441098,26.152401,Multiplex


Checking how many venues were returned for each neighborhood:

In [47]:
bucharest_venues_count = bucharest_venues.groupby('Neighborhood').count()
bucharest_venues_count.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
13 septembrie,13,13,13,13,13,13
Aviatorilor,29,29,29,29,29,29
Berceni,12,12,12,12,12,12
Bucurestii Noi,7,7,7,7,7,7
Centrul Vechi,30,30,30,30,30,30


Let's select the top 10 neighborhoods in terms of number of venues.

In [168]:
bucharest_top=bucharest_venues_count.sort_values(by = ['Venue'], ascending=False)
bucharest_top.head(10)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Piata Romana,30,30,30,30,30,30
Piata Sudului,30,30,30,30,30,30
Piata Universitatii,30,30,30,30,30,30
Pantelimon,30,30,30,30,30,30
Centrul Vechi,30,30,30,30,30,30
Unirii,30,30,30,30,30,30
Stefan cel mare,30,30,30,30,30,30
Dorobanti,30,30,30,30,30,30
Timpuri Noi,30,30,30,30,30,30


In [174]:
bucharest_top.index

Index(['Piata Romana', 'Piata Sudului', 'Piata Universitatii', 'Pantelimon',
       'Centrul Vechi', 'Unirii', 'Stefan cel mare', 'Dorobanti',
       'Timpuri Noi', 'Aviatorilor', 'Primaverii', 'Muncii', 'Crangasi',
       'Iancului', 'Obor', 'Titan', 'Tineretului', 'Cotroceni', 'Dristor',
       'Grozavesti', 'Ion Creanga', 'Drumul Taberei', 'Vacaresti',
       '13 septembrie', 'Gara de Nord', 'Grivita', 'Berceni', 'Victoriei',
       'Vitan', 'Ghencea', 'Floreasca', 'Colentina', 'Militari',
       'Bucurestii Noi', 'Oltenitei', 'Pipera', 'Rahova', 'Giulesti',
       'Chitila', 'Piata Resita', 'Straulesti', 'Ferentari'],
      dtype='object', name='Neighborhood')

In [177]:
bucharest_venues_top_df=bucharest_venues[bucharest_venues['Neighborhood'] in bucharest_top.index]

TypeError: 'Series' objects are mutable, thus they cannot be hashed

Let's see how many unique venut categories are there:

In [57]:
print('There are {} unique categories.'.format(len(bucharest_venues['Venue Category'].unique())))

There are 164 unique categories.


#### Analysing each neighborhood

In [58]:
# one hot encoding
bucharest_onehot = pd.get_dummies(bucharest_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bucharest_onehot['Neighborhood'] = bucharest_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bucharest_onehot.columns[-1]] + list(bucharest_onehot.columns[:-1])
bucharest_onehot = bucharest_onehot[fixed_columns]

bucharest_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,BBQ Joint,...,Theme Park Ride / Attraction,Toy / Game Store,Train Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store
0,Pantelimon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pantelimon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Pantelimon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Pantelimon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pantelimon,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
bucharest_onehot.shape

(723, 165)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [61]:
bucharest_grouped=bucharest_onehot.groupby('Neighborhood').mean().reset_index()
bucharest_grouped

,Neighborhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,BBQ Joint,...,Theme Park Ride / Attraction,Toy / Game Store,Train Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Women's Store
0,13 septembrie,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
1,Aviatorilor,0.000000,0.034483,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
2,Berceni,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
3,Bucurestii Noi,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
4,Centrul Vechi,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.0000,0.066667,0.000000
5,Chitila,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
6,Colentina,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
7,Cotroceni,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.050000,0.000000
8,Crangasi,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
9,Dorobanti,0.000000,0.000000,0.033333,0.000000,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.033333,0.000000


In [62]:
bucharest_grouped.shape

(42, 165)

Print each neighborhood along with the top 5 most common venues

In [65]:
num_top_venues = 5

for hood in bucharest_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bucharest_grouped[bucharest_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----13 septembrie----
                venue  freq
0         Supermarket  0.15
1  Salon / Barbershop  0.15
2                 Bar  0.15
3        Concert Hall  0.08
4              Lounge  0.08


----Aviatorilor----
                       venue  freq
0         Italian Restaurant  0.10
1                     Bakery  0.07
2                      Hotel  0.07
3   Mediterranean Restaurant  0.07
4  Middle Eastern Restaurant  0.07


----Berceni----
                venue  freq
0                 Gym  0.17
1                Park  0.17
2       Grocery Store  0.08
3          Restaurant  0.08
4  Italian Restaurant  0.08


----Bucurestii Noi----
          venue  freq
0       Theater  0.14
1  Dessert Shop  0.14
2   Supermarket  0.14
3      Pharmacy  0.14
4           Gym  0.14


----Centrul Vechi----
            venue  freq
0     Coffee Shop  0.13
1       Roof Deck  0.07
2        Beer Bar  0.07
3        Wine Bar  0.07
4  Ice Cream Shop  0.07


----Chitila----
                     venue  freq
0              S

Let's copy the above result inside a pandas dataframe.

A function to sort the venues in descending order.

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bucharest_grouped['Neighborhood']

for ind in np.arange(bucharest_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bucharest_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,13 septembrie,Bar,Supermarket,Salon / Barbershop,Concert Hall,Lounge,Romanian Restaurant,Pizza Place,Park,Gym / Fitness Center,Cosmetics Shop
1,Aviatorilor,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant,Mediterranean Restaurant,Hotel,Café,Bakery,Supermarket,Gym / Fitness Center
2,Berceni,Gym,Park,Fountain,Grocery Store,Restaurant,Italian Restaurant,Bakery,Lebanese Restaurant,Supermarket,Eastern European Restaurant
3,Bucurestii Noi,Bakery,Pharmacy,Dessert Shop,Gym,Supermarket,Theater,Shop & Service,Electronics Store,Flea Market,Fast Food Restaurant
4,Centrul Vechi,Coffee Shop,Roof Deck,Historic Site,Wine Bar,Beer Bar,Hotel,Ice Cream Shop,Bookstore,Tattoo Parlor,Chocolate Shop
5,Chitila,Health & Beauty Service,Cosmetics Shop,Lounge,Supermarket,Casino,Women's Store,Eye Doctor,Food & Drink Shop,Flea Market,Fast Food Restaurant
6,Colentina,Spa,Rental Car Location,Fast Food Restaurant,Hotel,Supermarket,Park,Food & Drink Shop,Eastern European Restaurant,Event Space,Flea Market
7,Cotroceni,Café,Bistro,Italian Restaurant,Pool,Tennis Court,Park,Restaurant,Electronics Store,Coffee Shop,Market
8,Crangasi,Café,Supermarket,Electronics Store,Pizza Place,Tennis Court,Fried Chicken Joint,Light Rail Station,Falafel Restaurant,Shoe Store,Farmers Market
9,Dorobanti,Café,Romanian Restaurant,Sushi Restaurant,Bakery,Spanish Restaurant,Cheese Shop,Sandwich Place,Cocktail Bar,Cosmetics Shop,Dessert Shop


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [107]:
# set number of clusters
kclusters = 5

bucharest_grouped_clustering = bucharest_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bucharest_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 0, 1, 1, 1], dtype=int32)

New dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [104]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bucharest_merged = df

# merge bucharest_grouped with bucharest dataframe to add latitude/longitude for each neighborhood
bucharest_merged = bucharest_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bucharest_merged.head() 

ValueError: columns overlap but no suffix specified: Index(['Cluster Labels', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue', '6th Most Common Venue',
       '7th Most Common Venue', '8th Most Common Venue',
       '9th Most Common Venue', '10th Most Common Venue'],
      dtype='object')

In [105]:
bucharest_merged.head() 

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pantelimon,44.443762,26.152474,1.0,Gym / Fitness Center,Burger Joint,Café,Department Store,Bakery,Taco Place,Lebanese Restaurant,Shopping Mall,Lounge,Bookstore
1,Pipera,44.504337,26.136806,1.0,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store,Italian Restaurant,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor
2,Ferentari,44.399308,26.079419,4.0,Doner Restaurant,Auto Garage,Women's Store,Eye Doctor,Fountain,Football Stadium,Food & Drink Shop,Flea Market,Fast Food Restaurant,Farmers Market
3,Bucurestii Noi,44.490262,26.032301,1.0,Bakery,Pharmacy,Dessert Shop,Gym,Supermarket,Theater,Shop & Service,Electronics Store,Flea Market,Fast Food Restaurant
4,Aviatorilor,44.459504,26.081038,1.0,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant,Mediterranean Restaurant,Hotel,Café,Bakery,Supermarket,Gym / Fitness Center


In [109]:
bucharest_merged.dropna(subset=['Cluster Labels'], axis=0, inplace=True)
bucharest_merged.reset_index(drop=True, inplace=True)
bucharest_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pantelimon,44.443762,26.152474,1.0,Gym / Fitness Center,Burger Joint,Café,Department Store,Bakery,Taco Place,Lebanese Restaurant,Shopping Mall,Lounge,Bookstore
1,Pipera,44.504337,26.136806,1.0,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store,Italian Restaurant,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor
2,Ferentari,44.399308,26.079419,4.0,Doner Restaurant,Auto Garage,Women's Store,Eye Doctor,Fountain,Football Stadium,Food & Drink Shop,Flea Market,Fast Food Restaurant,Farmers Market
3,Bucurestii Noi,44.490262,26.032301,1.0,Bakery,Pharmacy,Dessert Shop,Gym,Supermarket,Theater,Shop & Service,Electronics Store,Flea Market,Fast Food Restaurant
4,Aviatorilor,44.459504,26.081038,1.0,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant,Mediterranean Restaurant,Hotel,Café,Bakery,Supermarket,Gym / Fitness Center


In [110]:
bucharest_merged['Cluster Labels']=bucharest_merged['Cluster Labels'].astype(int)

In [111]:
bucharest_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pantelimon,44.443762,26.152474,1,Gym / Fitness Center,Burger Joint,Café,Department Store,Bakery,Taco Place,Lebanese Restaurant,Shopping Mall,Lounge,Bookstore
1,Pipera,44.504337,26.136806,1,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store,Italian Restaurant,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor
2,Ferentari,44.399308,26.079419,4,Doner Restaurant,Auto Garage,Women's Store,Eye Doctor,Fountain,Football Stadium,Food & Drink Shop,Flea Market,Fast Food Restaurant,Farmers Market
3,Bucurestii Noi,44.490262,26.032301,1,Bakery,Pharmacy,Dessert Shop,Gym,Supermarket,Theater,Shop & Service,Electronics Store,Flea Market,Fast Food Restaurant
4,Aviatorilor,44.459504,26.081038,1,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant,Mediterranean Restaurant,Hotel,Café,Bakery,Supermarket,Gym / Fitness Center


Visualize the resulting clusters:

In [114]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cm

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bucharest_merged['Latitude'], bucharest_merged['Longitude'], bucharest_merged['Neighborhood'], bucharest_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine clusters

First Cluster

In [137]:
bucharest_merged_cluster1=bucharest_merged[bucharest_merged['Cluster Labels']==0]
bucharest_merged_cluster1.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Centrul Vechi,44.431510,26.100200,0,Coffee Shop,Roof Deck,Historic Site,Wine Bar,Beer Bar,Hotel,Ice Cream Shop,Bookstore,Tattoo Parlor,Chocolate Shop
7,Colentina,44.466426,26.151016,0,Spa,Rental Car Location,Fast Food Restaurant,Hotel,Supermarket,Park,Food & Drink Shop,Eastern European Restaurant,Event Space,Flea Market
11,Drumul Taberei,44.420321,26.033693,0,Coffee Shop,Restaurant,Farmers Market,Park,Department Store,Pizza Place,Romanian Restaurant,Sandwich Place,Café,Shopping Mall
12,Floreasca,44.468005,26.103123,0,Pool,Lake,Eastern European Restaurant,Skating Rink,Mini Golf,French Restaurant,Lounge,Light Rail Station,Botanical Garden,Hotel
13,Gara de Nord,44.446882,26.075925,0,Hotel,Pub,Salon / Barbershop,Café,Pizza Place,Middle Eastern Restaurant,Grocery Store,Boarding House,Farmers Market,Bakery


Cluster 2

In [128]:
bucharest_merged_cluster2=bucharest_merged[bucharest_merged['Cluster Labels']==1]
bucharest_merged_cluster2.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Pantelimon,44.443762,26.152474,1,Gym / Fitness Center,Burger Joint,Café,Department Store,Bakery,Taco Place,Lebanese Restaurant,Shopping Mall,Lounge,Bookstore
1,Pipera,44.504337,26.136806,1,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store,Italian Restaurant,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor
3,Bucurestii Noi,44.490262,26.032301,1,Bakery,Pharmacy,Dessert Shop,Gym,Supermarket,Theater,Shop & Service,Electronics Store,Flea Market,Fast Food Restaurant
4,Aviatorilor,44.459504,26.081038,1,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant,Mediterranean Restaurant,Hotel,Café,Bakery,Supermarket,Gym / Fitness Center
6,Dristor,44.418970,26.141623,1,Supermarket,Restaurant,Pizza Place,Cupcake Shop,Gastropub,Doner Restaurant,Coffee Shop,Drugstore,Café,Farmers Market


Cluster 3

In [129]:
bucharest_merged_cluster3=bucharest_merged[bucharest_merged['Cluster Labels']==2]
bucharest_merged_cluster3.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Oltenitei,44.386663,26.13794,2,Pizza Place,Supermarket,Soccer Field,Women's Store,Electronics Store,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Eye Doctor


Cluster 4

In [130]:
bucharest_merged_cluster4=bucharest_merged[bucharest_merged['Cluster Labels']==3]
bucharest_merged_cluster4.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Giulesti,44.464324,26.035618,3,Climbing Gym,Rest Area,Light Rail Station,Lake,Women's Store,Event Space,Food & Drink Shop,Flea Market,Fast Food Restaurant,Farmers Market
17,Grivita,44.459719,26.067321,3,Light Rail Station,American Restaurant,Bakery,Cupcake Shop,Park,Electronics Store,Boutique,Garden Center,Pizza Place,Grocery Store
22,Rahova,44.400135,26.053329,3,Light Rail Station,Supermarket,Grocery Store,Women's Store,Event Space,Food & Drink Shop,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant


Cluster 5

In [131]:
bucharest_merged_cluster5=bucharest_merged[bucharest_merged['Cluster Labels']==4]
bucharest_merged_cluster5.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Ferentari,44.399308,26.079419,4,Doner Restaurant,Auto Garage,Women's Store,Eye Doctor,Fountain,Football Stadium,Food & Drink Shop,Flea Market,Fast Food Restaurant,Farmers Market


We conclude the most popular neighborhoods are the ones included inside the first 2 clusters. The result seems trustworthy, considering the last cluster contains an unsafe neighborhood.

Considering the person who wants to buy an apartment is not a big fan of cooking, we want to choose neighborhoods where there are many restaurants.   Also, a supermarket close by, a gym and a park are highly recommended, therefore, let's select the best neighborhoods.

In [185]:
cluster1_df=bucharest_merged_cluster1[['Neighborhood','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue']]
cluster1_df

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
5,Centrul Vechi,Coffee Shop,Roof Deck,Historic Site,Wine Bar
7,Colentina,Spa,Rental Car Location,Fast Food Restaurant,Hotel
11,Drumul Taberei,Coffee Shop,Restaurant,Farmers Market,Park
12,Floreasca,Pool,Lake,Eastern European Restaurant,Skating Rink
13,Gara de Nord,Hotel,Pub,Salon / Barbershop,Café
20,Obor,Coffee Shop,BBQ Joint,Noodle House,Burger Joint
21,Primaverii,Coffee Shop,Italian Restaurant,Mediterranean Restaurant,Bistro
23,Straulesti,Dessert Shop,Hotel,Burger Joint,Grocery Store
26,Unirii,Coffee Shop,Gym / Fitness Center,Ice Cream Shop,Bagel Shop
28,Victoriei,Pizza Place,Event Space,Bus Station,Hotel


In [188]:
cluster1_df2=cluster1_df.set_index(cluster1_df['Neighborhood'])
cluster1_df2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,,
Centrul Vechi,Centrul Vechi,Coffee Shop,Roof Deck,Historic Site,Wine Bar
Colentina,Colentina,Spa,Rental Car Location,Fast Food Restaurant,Hotel
Drumul Taberei,Drumul Taberei,Coffee Shop,Restaurant,Farmers Market,Park
Floreasca,Floreasca,Pool,Lake,Eastern European Restaurant,Skating Rink
Gara de Nord,Gara de Nord,Hotel,Pub,Salon / Barbershop,Café
Obor,Obor,Coffee Shop,BBQ Joint,Noodle House,Burger Joint
Primaverii,Primaverii,Coffee Shop,Italian Restaurant,Mediterranean Restaurant,Bistro
Straulesti,Straulesti,Dessert Shop,Hotel,Burger Joint,Grocery Store
Unirii,Unirii,Coffee Shop,Gym / Fitness Center,Ice Cream Shop,Bagel Shop


In [189]:
cluster1_df2.drop('Neighborhood', axis=1,inplace=True)
cluster1_df2

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,
Centrul Vechi,Coffee Shop,Roof Deck,Historic Site,Wine Bar
Colentina,Spa,Rental Car Location,Fast Food Restaurant,Hotel
Drumul Taberei,Coffee Shop,Restaurant,Farmers Market,Park
Floreasca,Pool,Lake,Eastern European Restaurant,Skating Rink
Gara de Nord,Hotel,Pub,Salon / Barbershop,Café
Obor,Coffee Shop,BBQ Joint,Noodle House,Burger Joint
Primaverii,Coffee Shop,Italian Restaurant,Mediterranean Restaurant,Bistro
Straulesti,Dessert Shop,Hotel,Burger Joint,Grocery Store
Unirii,Coffee Shop,Gym / Fitness Center,Ice Cream Shop,Bagel Shop


In [224]:
restaurant_search1=cluster1_df2[cluster1_df2['1st Most Common Venue'].str.contains("Restaurant") | cluster1_df2['2nd Most Common Venue'].str.contains("Restaurant") | cluster1_df2['3rd Most Common Venue'].str.contains("Restaurant")]
restaurant_search1

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,
Colentina,Spa,Rental Car Location,Fast Food Restaurant,Hotel
Drumul Taberei,Coffee Shop,Restaurant,Farmers Market,Park
Floreasca,Pool,Lake,Eastern European Restaurant,Skating Rink
Primaverii,Coffee Shop,Italian Restaurant,Mediterranean Restaurant,Bistro
Piata Universitatii,Hotel,Italian Restaurant,Coffee Shop,Theater


We see these are these are the neighborhoods inside cluster 1 which contain restaurants as their top 3 most common venue.

Out of these locations, let's select those which also contain a supermarket as a top venue.

In [226]:
supermarket_search1=restaurant_search1[restaurant_search1['1st Most Common Venue'].str.contains("Supermarket") | restaurant_search1['2nd Most Common Venue'].str.contains("Supermarket") | restaurant_search1['3rd Most Common Venue'].str.contains("Supermarket") | restaurant_search1['4th Most Common Venue'].str.contains("Supermarket")]
supermarket_search1

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,


We see that, despite having restaurants close by, we don't have a supermarket as a top venue in neither of these neighborhoods.

Let's have a look inside cluster 2 as well

In [214]:
cluster2_df=bucharest_merged_cluster2[['Neighborhood','1st Most Common Venue','2nd Most Common Venue','3rd Most Common Venue','4th Most Common Venue']]
cluster2_df.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Pantelimon,Gym / Fitness Center,Burger Joint,Café,Department Store
1,Pipera,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store
3,Bucurestii Noi,Bakery,Pharmacy,Dessert Shop,Gym
4,Aviatorilor,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant
6,Dristor,Supermarket,Restaurant,Pizza Place,Cupcake Shop


In [216]:
cluster2_df2=cluster2_df.set_index(cluster2_df['Neighborhood'])
cluster2_df2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,,
Pantelimon,Pantelimon,Gym / Fitness Center,Burger Joint,Café,Department Store
Pipera,Pipera,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store
Bucurestii Noi,Bucurestii Noi,Bakery,Pharmacy,Dessert Shop,Gym
Aviatorilor,Aviatorilor,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant
Dristor,Dristor,Supermarket,Restaurant,Pizza Place,Cupcake Shop
Cotroceni,Cotroceni,Café,Bistro,Italian Restaurant,Pool
Crangasi,Crangasi,Café,Supermarket,Electronics Store,Pizza Place
Dorobanti,Dorobanti,Café,Romanian Restaurant,Sushi Restaurant,Bakery
Ghencea,Ghencea,Bus Station,Gym,Supermarket,Pub


In [217]:
cluster2_df2.drop('Neighborhood', axis=1,inplace=True)
cluster2_df2

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,
Pantelimon,Gym / Fitness Center,Burger Joint,Café,Department Store
Pipera,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store
Bucurestii Noi,Bakery,Pharmacy,Dessert Shop,Gym
Aviatorilor,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant
Dristor,Supermarket,Restaurant,Pizza Place,Cupcake Shop
Cotroceni,Café,Bistro,Italian Restaurant,Pool
Crangasi,Café,Supermarket,Electronics Store,Pizza Place
Dorobanti,Café,Romanian Restaurant,Sushi Restaurant,Bakery
Ghencea,Bus Station,Gym,Supermarket,Pub


In [227]:
restaurant_search2=cluster2_df2[cluster2_df2['1st Most Common Venue'].str.contains("Restaurant") | cluster2_df2['2nd Most Common Venue'].str.contains("Restaurant") | cluster2_df2['3rd Most Common Venue'].str.contains("Restaurant")]
restaurant_search2

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,
Pipera,Cupcake Shop,Seafood Restaurant,Flea Market,Grocery Store
Aviatorilor,Italian Restaurant,Restaurant,Romanian Restaurant,Middle Eastern Restaurant
Dristor,Supermarket,Restaurant,Pizza Place,Cupcake Shop
Cotroceni,Café,Bistro,Italian Restaurant,Pool
Dorobanti,Café,Romanian Restaurant,Sushi Restaurant,Bakery
Grozavesti,Pub,Romanian Restaurant,Restaurant,Bus Station
Militari,Supermarket,Middle Eastern Restaurant,Restaurant,Park
Timpuri Noi,Italian Restaurant,Café,Turkish Restaurant,Restaurant
Tineretului,Café,Supermarket,Romanian Restaurant,Italian Restaurant


We see more results here, but out of them, let's select those which have also a supermarket as a common venue.

In [228]:
supermarket_search2=restaurant_search2[restaurant_search2['1st Most Common Venue'].str.contains("Supermarket") | restaurant_search2['2nd Most Common Venue'].str.contains("Supermarket") | restaurant_search2['3rd Most Common Venue'].str.contains("Supermarket") | restaurant_search2['4th Most Common Venue'].str.contains("Supermarket")]
supermarket_search2

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,
Dristor,Supermarket,Restaurant,Pizza Place,Cupcake Shop
Militari,Supermarket,Middle Eastern Restaurant,Restaurant,Park
Tineretului,Café,Supermarket,Romanian Restaurant,Italian Restaurant
Iancului,Pharmacy,Gym,Eastern European Restaurant,Supermarket


Considering we have more than one returned neighborhood, let's select one which contain as a top venue the third client interest, which is a gym.

In [229]:
gym_search=supermarket_search2[supermarket_search2['1st Most Common Venue'].str.contains("Gym") | supermarket_search2['2nd Most Common Venue'].str.contains("Gym") | supermarket_search2['3rd Most Common Venue'].str.contains("Gym") | supermarket_search2['4th Most Common Venue'].str.contains("Gym")]
gym_search

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
Neighborhood,,,,
Iancului,Pharmacy,Gym,Eastern European Restaurant,Supermarket


Therefore, we decide the best place considering our client's needs is Iancului, having the biggest number of restaurants, sueprmarkets and gym nearby.

Now, let's make a dataframe containing the venues of Iancului neighborhood.

In [240]:
iancului_neighborhood=bucharest_venues[bucharest_venues['Neighborhood']=='Iancului']
iancului_neighborhood.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
542,Iancului,44.441871,26.138562,Bery Fitness & Spa,44.443772,26.138594,Gym
543,Iancului,44.441871,26.138562,Cofetăria Alice,44.438933,26.135521,Dessert Shop
544,Iancului,44.441871,26.138562,Sandow Fitness Club,44.440820,26.139551,Gym
545,Iancului,44.441871,26.138562,Calif - Califatul Iancului,44.441317,26.134170,Kebab Restaurant
546,Iancului,44.441871,26.138562,Golden Kebap,44.445198,26.136019,Turkish Restaurant
547,Iancului,44.441871,26.138562,I'Ladrino,44.442386,26.141936,Italian Restaurant
548,Iancului,44.441871,26.138562,Mega Image,44.441384,26.135051,Supermarket
549,Iancului,44.441871,26.138562,Parcul Ciurea,44.438020,26.140621,Park
550,Iancului,44.441871,26.138562,Trenta Pizza,44.444970,26.137200,Pizza Place
551,Iancului,44.441871,26.138562,Mega Image,44.438271,26.138960,Supermarket


In order to be able to count the number of restaurants regardless of their type, let's name any type of restaurant as "Restaurant"

In [260]:
iancului_neighborhood['Venue Category'] = np.where(iancului_neighborhood['Venue Category'].str.contains('Restaurant'), 'Restaurant', iancului_neighborhood['Venue Category'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [261]:
iancului_neighborhood.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category Update
542,Iancului,44.441871,26.138562,Bery Fitness & Spa,44.443772,26.138594,Gym,Gym
543,Iancului,44.441871,26.138562,Cofetăria Alice,44.438933,26.135521,Dessert Shop,Dessert Shop
544,Iancului,44.441871,26.138562,Sandow Fitness Club,44.440820,26.139551,Gym,Gym
545,Iancului,44.441871,26.138562,Calif - Califatul Iancului,44.441317,26.134170,Restaurant,Restaurant
546,Iancului,44.441871,26.138562,Golden Kebap,44.445198,26.136019,Restaurant,Restaurant
547,Iancului,44.441871,26.138562,I'Ladrino,44.442386,26.141936,Restaurant,Restaurant
548,Iancului,44.441871,26.138562,Mega Image,44.441384,26.135051,Supermarket,Supermarket
549,Iancului,44.441871,26.138562,Parcul Ciurea,44.438020,26.140621,Park,Park
550,Iancului,44.441871,26.138562,Trenta Pizza,44.444970,26.137200,Pizza Place,Pizza Place
551,Iancului,44.441871,26.138562,Mega Image,44.438271,26.138960,Supermarket,Supermarket


In [264]:
iancului_neighborhood.drop('Venue Category Update',1, inplace=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [265]:
iancului_neighborhood.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
542,Iancului,44.441871,26.138562,Bery Fitness & Spa,44.443772,26.138594,Gym
543,Iancului,44.441871,26.138562,Cofetăria Alice,44.438933,26.135521,Dessert Shop
544,Iancului,44.441871,26.138562,Sandow Fitness Club,44.440820,26.139551,Gym
545,Iancului,44.441871,26.138562,Calif - Califatul Iancului,44.441317,26.134170,Restaurant
546,Iancului,44.441871,26.138562,Golden Kebap,44.445198,26.136019,Restaurant
547,Iancului,44.441871,26.138562,I'Ladrino,44.442386,26.141936,Restaurant
548,Iancului,44.441871,26.138562,Mega Image,44.441384,26.135051,Supermarket
549,Iancului,44.441871,26.138562,Parcul Ciurea,44.438020,26.140621,Park
550,Iancului,44.441871,26.138562,Trenta Pizza,44.444970,26.137200,Pizza Place
551,Iancului,44.441871,26.138562,Mega Image,44.438271,26.138960,Supermarket


In [266]:
iancului_neighborhood['Venue Category'].value_counts()

Restaurant              6
Park                    2
Pharmacy                2
Supermarket             2
Gym                     2
Farmers Market          1
Café                    1
Dessert Shop            1
Plaza                   1
Diner                   1
Bagel Shop              1
Pizza Place             1
Convenience Store       1
Gym / Fitness Center    1
Name: Venue Category, dtype: int64

In [267]:
iancului_neighborhood['Venue Category'] = np.where(iancului_neighborhood['Venue Category'].str.contains('Gym'), 'Gym', iancului_neighborhood['Venue Category'])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [268]:
iancului_neighborhood['Venue Category'].value_counts()

Restaurant           6
Gym                  3
Park                 2
Pharmacy             2
Supermarket          2
Farmers Market       1
Café                 1
Dessert Shop         1
Plaza                1
Bagel Shop           1
Pizza Place          1
Convenience Store    1
Diner                1
Name: Venue Category, dtype: int64

Considering these results, we conclude that Iancului is a good neighborhoods for our client's needs, as it contains 6 restaurants, 2 supermarkets and 3 gyms in less than 500 metres.